### States
States is a list of all possible states in given environment.

In [1]:
import numpy as np
states = ['Class 1', 'Class 2', 'Class 3', 'Facebook', 'Stop']

### Class Action
Action class represents an singe action that agent can take.
* to - represents next state when agent will take this action
* p - probability of this action. Default it is 1, but if single Action can lead to multiple states p is probability that we end up in to state

In [2]:
class Action:
    def __init__(self, to, p=1):
        self.to = to
        self.p = p
    
    def __repr__(self):
        return "(to: {}, p: {})"\
            .format(self.to, self.p)

### State actions
State actions is an dictionary where key is a state name and a value is all possible action that can be taken from this state.
Actions for state are also represented as dictionaries. Key is a action name and value is a list of Action classes

In [3]:
state_actions = {
    'Class 1': {
        'Study': [Action('Class 2')],
        'Facebook': [Action('Facebook')]
    },
    'Class 2': {
        'Study': [Action('Class 3')],
        'Sleep': [Action('Stop')]
    },
    'Class 3': {
        'Study': [Action('Stop')],
        'Pub': [
            Action('Class 3', 0.4),
            Action('Class 2', 0.4),
            Action('Class 1', 0.2)
        ]
    },
    'Facebook': {
        'Facebook': [Action('Facebook')],
        'Quit': [Action('Class 1')]
    },
    'Stop': {}
}

### State action reward
This is dictionary that represends a reward for taking an action A when we are in state S.
Key is state name and value is a dictionary with pairs: action name -> reward

In [4]:
state_actions_reward = {
    'Class 1': {
        'Study': -2,
        'Facebook': -1
    },
    'Class 2': {
        'Study': -2,
        'Sleep': 0
    },
    'Class 3': {
        'Study': 10,
        'Pub': 1
    },
    'Facebook': {
        'Facebook': -1,
        'Quit': 0
    },
    'Stop': {}
}

### Terminal states
List of states that ends an episode

In [5]:
terminal_states = ['Stop']

### Value function and Policy
**Value function V** is dictionary that for each state S stores value of this state. That is, states with higer value are simply better.

**Policy** is another dictionary that for state S stores an action A that agent should take in this state.

Our goal is to find **optimal policy** - policy that will maximalize reward.

**Discount** is value from 0 to 1 inclusive. Each next step is discounted with **discount** factor. If closer to 0 agent will prefer imidiete rewards and will not care about rewards in far future. When closer to 1 agent will prefer to take actions that will grant him hiher rewards in the futrue.

In [6]:
V = dict((s, 0) for s in states)
Policy = dict((s, None) for s in states)
discount = 0.8

### Optimum class
Optimum class represents optimal value that maximaze **state value** for given state.
* **value** is value of given state
* **action** is action that should be taken in given state

In [7]:
class Optimum:
    def __init__(self, value, action):
        self.value = value
        self.action = action

### v(state) function
**v(state) function** findes Optimum for given state.
It is calculated as follows:
1. For each possible action a for given state s calculate:
  V(s) = r + d * SUM(p * V(ns)) where:
  * V(s) is value of state s
  * r is reward for taking action a
  * SUM is sum for all possible output states of taking action a
  * p is probability of going to state ns after taking action a
  * ns is state that result of taking action a
  * V(ns) is value of state ns
2. From above calculation take maximum and this is new value for state s.

In [8]:
# Optimal value function for state
def v(state):
    values = []
    for action in state_actions[state].keys():
        reward = state_actions_reward[state][action]
        future = discount * sum([act.p * V[act.to] for act in state_actions[state][action]])
        values.append(Optimum(reward + future, action))
        
    optimum = max(values, key=lambda item: item.value)
    return optimum

### Value interation
This is method of calculating Bellman Equation using dynamic programming paradigm.
It takes following parameters:
* n - number of iterations
* theta - threshold of convergence
* verobse - display iterations
Method calculate optimal state value and policy for each state. It stops after:
* Iterating n times, or
* Achieving convergence of state-value

In [9]:
def value_iteration(n=10, theta=0.02, verbose=False):
    for _ in range(n):
        old_V = np.array([x for x in V.values()])
        for state in states:
            if state in terminal_states:
                continue
            opt = v(state)
            Policy[state] = opt.action
            V[state] = opt.value
        new_V = np.array([x for x in V.values()])
        delta = abs(max(old_V-new_V))
        if verbose: 
            print("Value_function: V={}".format(V))
        if delta < theta:
            break

### Run and display results

In [10]:
discount = 1
value_iteration()
print(V)
print(Policy)

{'Class 1': 6, 'Class 2': 8, 'Class 3': 10, 'Facebook': 6, 'Stop': 0}
{'Class 1': 'Study', 'Class 2': 'Study', 'Class 3': 'Study', 'Facebook': 'Quit', 'Stop': None}
